In [1]:
import requests
import json
import os
from datetime import datetime

# Mapping of cid to city names
cid_to_city = {
    "P48": "Chennai (Madras)",
    "P147": "New Delhi",
    "P139": "Mumbai (Bombay)",
    "P100": "Kolkata",
    "P85": "Hyderabad"
}

# Define the column index to print (0-based index)
column_index_to_print = 2  # 2 is column number for "Interview Required Petition-Based Temporary Workers (H, L, O, P, Q)"

# Define the base URL and the possible values for the cid parameter
base_url = "https://travel.state.gov/content/travel/resources/database/database.getVisaWaitTimes.html"
cid_values = ["P48", "P147", "P139", "P100", "P85"]
aid_value = "VisaWaitTimesHomePage"

# Function to get the visa wait times for a given cid and return a specified column
def get_visa_wait_times_column(cid, column_index):
    params = {
        "cid": cid,
        "aid": aid_value
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        response_text = response.text
        columns = response_text.split('|')
        
        # Return the specified column if it exists, otherwise return "No data"
        if column_index < len(columns):
            return columns[column_index].strip() if columns[column_index].strip() != '' else "Current"
        else:
            return "No data"
    else:
        return f"Failed to fetch data for cid={cid}, status code: {response.status_code}"

# Get the current time for the last_refreshed field
last_refreshed = datetime.now().strftime("%b %d, %Y %I:%M %p")

# Prepare data for JSON
data = {
    "last_refreshed": last_refreshed,
    "index 0": {
        "Nonimmigrant Visa Type": "Interview Required Petition-Based Temporary Workers (H, L, O, P, Q)",
        "records": []
    }
}

# Loop through each cid value, call the URL, and collect the specified column with city name
for cid in cid_values:
    column_value = get_visa_wait_times_column(cid, column_index_to_print)
    city_name = cid_to_city.get(cid, "Unknown City")
    data["index 0"]["records"] += [{city_name: column_value}]

# Define the file path
file_path = r"C:\Users\singh\dataneb\files\AppointmentWaitTimeIndia_HLOPQ.json"

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Save data to JSON file
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"Data has been saved to {file_path}")


Data has been saved to C:\Users\singh\dataneb\files\AppointmentWaitTimeIndia_HLOPQ.json
